# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time 

# Import API key
from config import api_key #congig is the name of the file and api_key is the name of the variable within the file

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities_data.csv")
weather_data #test

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Daru,-9.0763,143.2092,79.34,87,100,10.07,PG,1611351610
1,Punta Arenas,-53.1500,-70.9167,57.20,33,40,21.85,CL,1611351610
2,Mīzan Teferī,6.9833,35.5833,66.09,58,86,2.73,ET,1611351610
3,Jamestown,42.0970,-79.2353,23.00,86,90,16.11,US,1611351423
4,Nikolskoye,59.7035,30.7861,35.60,87,40,15.66,RU,1611351611
...,...,...,...,...,...,...,...,...,...
570,Kangaatsiaq,68.3065,-53.4641,13.42,85,48,7.78,GL,1611351750
571,La Grande,45.3246,-118.0877,37.00,87,99,3.27,US,1611351750
572,Tombouctou,20.0000,-3.0000,66.74,20,47,9.95,ML,1611351751
573,Ndola,-12.9587,28.6366,64.20,98,100,3.87,ZM,1611351751


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=api_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [9]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

#max_intensity

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 65 and 75
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 65) & (weather_data["Max Temp"] <= 75)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,Hermanus,-34.4187,19.2345,66.00,93,0,3.06,ZA,1611351611
16,Bredasdorp,-34.5322,20.0403,71.60,88,0,2.30,ZA,1611351614
31,Hong Kong,22.2855,114.1577,66.00,89,0,6.60,HK,1611351533
105,Umluj,25.0213,37.2685,70.97,40,0,4.79,SA,1611351636
112,Salalah,17.0151,54.0924,73.40,25,0,3.44,OM,1611351637
370,Wanning,18.8003,110.3967,66.06,89,0,3.44,CN,1611351421
381,Olpād,21.3333,72.7500,67.14,62,0,6.71,IN,1611351705
467,Taoudenni,22.6783,-3.9836,65.79,20,0,9.89,ML,1611351724
476,Broken Hill,-31.9500,141.4333,71.60,16,0,3.44,AU,1611351725
551,Kibala,9.1104,18.3463,73.40,43,0,2.77,TD,1611351746


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Create a the hotel dataframe 
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,Hermanus,ZA,-34.4187,19.2345,
16,Bredasdorp,ZA,-34.5322,20.0403,
31,Hong Kong,HK,22.2855,114.1577,
105,Umluj,SA,25.0213,37.2685,
112,Salalah,OM,17.0151,54.0924,
370,Wanning,CN,18.8003,110.3967,
381,Olpād,IN,21.3333,72.7500,
467,Taoudenni,ML,22.6783,-3.9836,
476,Broken Hill,AU,-31.9500,141.4333,
551,Kibala,TD,9.1104,18.3463,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : api_key}

In [13]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 6: Hermanus.
Missing field/result... skipping.
------------
Retrieving Results for Index 16: Bredasdorp.
Missing field/result... skipping.
------------
Retrieving Results for Index 31: Hong Kong.
Missing field/result... skipping.
------------
Retrieving Results for Index 105: Umluj.
Missing field/result... skipping.
------------
Retrieving Results for Index 112: Salalah.
Missing field/result... skipping.
------------
Retrieving Results for Index 370: Wanning.
Missing field/result... skipping.
------------
Retrieving Results for Index 381: Olpād.
Missing field/result... skipping.
------------
Retrieving Results for Index 467: Taoudenni.
Missing field/result... skipping.
------------
Retrieving Results for Index 476: Broken Hill.
Missing field/result... skipping.
------------
Retrieving Results for Index 551: Kibala.
Missing field/result... skipping.
------------
-------End of Search-------


In [24]:
hotel_df #display the new dataframe 

,City,Country,Lat,Lng,Hotel Name
6,Hermanus,ZA,-34.4187,19.2345,
16,Bredasdorp,ZA,-34.5322,20.0403,
31,Hong Kong,HK,22.2855,114.1577,
105,Umluj,SA,25.0213,37.2685,
112,Salalah,OM,17.0151,54.0924,
370,Wanning,CN,18.8003,110.3967,
381,Olpād,IN,21.3333,72.7500,
467,Taoudenni,ML,22.6783,-3.9836,
476,Broken Hill,AU,-31.9500,141.4333,
551,Kibala,TD,9.1104,18.3463,


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))